In [231]:
import pandas as pd
import numpy as np
from random import randint

In [232]:
dataset = pd.read_csv('dataset_dbscan.csv', delimiter=';')

In [305]:
array_dataset = dataset.drop('No',axis=1)
array_dataset = array_dataset.values
pd.DataFrame({'Temperatur':array_dataset[:,0],'Kelembapan':array_dataset[:,1],'Cluster':array_dataset[:,2]})

,Temperatur,Kelembapan,Cluster
0,85,85,0
1,80,90,0
2,65,70,0
3,72,95,0
4,83,78,0
5,60,75,0
6,78,85,0
7,64,65,0
8,69,70,0
9,75,88,0


In [306]:
array_dataset

array([[85, 85,  0],
       [80, 90,  0],
       [65, 70,  0],
       [72, 95,  0],
       [83, 78,  0],
       [60, 75,  0],
       [78, 85,  0],
       [64, 65,  0],
       [69, 70,  0],
       [75, 88,  0]], dtype=int64)

## Calculate Distance

In [307]:
#calculate distance with Manhattan
def cal_distance(data1, data2):
    temp1 = data1[0]
    temp2 = data2[0]
    humd1 = data1[1]
    humd2 = data2[1]
    distance = abs(temp1-temp2)+abs(humd1-humd2)
    return distance

In [308]:
#test method calculate distance
dataTest = []
for i in range(0,len(array_dataset)):
    print (array_dataset[i],cal_distance(array_dataset[2],array_dataset[i]))

[85 85  0] 35
[80 90  0] 35
[65 70  0] 0
[72 95  0] 32
[83 78  0] 26
[60 75  0] 10
[78 85  0] 28
[64 65  0] 6
[69 70  0] 4
[75 88  0] 28


## Define Neighbors & Cluster

In [247]:
def get_neighbors(dataset,cp):
    index_neighbors = []
    eps = 15
    
    for i in range(len(dataset)):
        count_distance = cal_distance(cp,dataset[i])
        if count_distance <= eps and count_distance != 0:
            index_neighbors.append(i)
            
    return index_neighbors

def define_cluster(dataset,neighbor_array,cluster,minPts):
    
    count=0
    while count <len(neighbor_array):
        next_point=neighbor_array[count]
        print('central point :',next_point)
        if dataset[next_point,-1]==-1:
            dataset[next_point,-1]=cluster
            
        elif dataset[next_point,-1]==0:
            dataset[next_point,-1]=cluster
        
            next_point_neighbor=get_neighbors(dataset[:,:-1],dataset[next_point,:-1])
            
            if len(next_point_neighbor)>=minPts:
                
                print("neigbor dari data",next_point,":", next_point_neighbor)
                neighbor_array = np.unique(np.concatenate((neighbor_array,next_point_neighbor)))
                print("all neighbor :",neighbor_array,"\n")
                #print(dataset,"\n")
                print(pd.DataFrame({'Temperatur':dataset[:,0],'Kelembapan':dataset[:,1],'Cluster':dataset[:,2]}),"\n")
        else:
            print("sudah memiliki cluster\n")
        count+=1
    print("Hasil Clustering",cluster,"\n",pd.DataFrame({'Temperatur':dataset[:,0],'Kelembapan':dataset[:,1],'Cluster':dataset[:,2]}),'\n\n============================================================\n')
        
def main_dbscan(dataset,minPts):
    cluster = 0
    central_point = 2
    
    for i in range(central_point,len(dataset)):
        if (dataset[i,-1]!=0):
            continue
        
        neighbor_array=get_neighbors(dataset[:,:-1],dataset[i,:-1])
        print("central point :",i)
        print("neighbor dari data",i,":",neighbor_array)
        if len(neighbor_array)<minPts:
            dataset[i,-1]=-1
            print(pd.DataFrame({'Temperatur':dataset[:,0],'Kelembapan':dataset[:,1],'Cluster':dataset[:,2]}),'\ndata noise\n')
        else :
            cluster+=1
            dataset[i,-1] = cluster
            #print("central point :",i)
            print(pd.DataFrame({'Temperatur':dataset[:,0],'Kelembapan':dataset[:,1],'Cluster':dataset[:,2]}),'\n')
            define_cluster(dataset,neighbor_array,cluster,minPts)
        
    return dataset

In [248]:
array_dataset = main_dbscan(array_dataset,3)

central point : 2
neighbor dari data 2 : [5, 7, 8]
   Temperatur  Kelembapan  Cluster
0          85          85        0
1          80          90        0
2          65          70        1
3          72          95        0
4          83          78        0
5          60          75        0
6          78          85        0
7          64          65        0
8          69          70        0
9          75          88        0 

central point : 5
neigbor dari data 5 : [2, 7, 8]
all neighbor : [2 5 7 8] 

   Temperatur  Kelembapan  Cluster
0          85          85        0
1          80          90        0
2          65          70        1
3          72          95        0
4          83          78        0
5          60          75        1
6          78          85        0
7          64          65        0
8          69          70        0
9          75          88        0 

central point : 5
sudah memiliki cluster

central point : 7
neigbor dari data 7 : [2, 5, 8]
all ne

In [249]:
result = pd.DataFrame({'Temperatur':array_dataset[:,0],'Kelembapan':array_dataset[:,1],'Cluster':array_dataset[:,2]})
result

,Temperatur,Kelembapan,Cluster
0,85,85,2
1,80,90,2
2,65,70,1
3,72,95,2
4,83,78,2
5,60,75,1
6,78,85,2
7,64,65,1
8,69,70,1
9,75,88,2
